# Root cause analysis Model

## Import Libraries

In [ ]:
import bnlearn as bn
import numpy as np
import pickle as pkl
import pandas as pd
import mlflow
import mlflow.sklearn

import sys
import numpy

import pyAgrum as gum
import pyAgrum.skbn as skbn
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.image as gph 
import pyAgrum.lib.bn2graph as bng
import pyAgrum.lib.bn_vs_bn as compare
import pyAgrum.lib.bn_vs_bn as compare
import pydotplus
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
numpy.set_printoptions(threshold=sys.maxsize)

%matplotlib inline

## Data Preparation

In [ ]:
#get process parameters, create a column representing whether a part is scrap or not
with open("./Data/causal_data_July.pkl", "rb") as f:
    obj = pkl.load(f)
    
df = pd.DataFrame(obj)

def label_race (row):
    if ((row['Durchmesser Min. Fübo 1'] >= 0.0008) | (row['Durchmesser Min. Fübo 1'] <= -0.0008)):
          return 1.0
    else:
        return 0.0
    
#df['is_scrap'] = df.apply (lambda row: label_race(row), axis=1)
df['Durchmesser Min. Fübo'] = df.apply (lambda row: label_race(row), axis=1)

df

In [ ]:
#get features of AE signals
with open("./Data/features_july.pkl", "rb") as f:
    obj = pkl.load(f)
    
features = pd.DataFrame(obj)
features

In [ ]:
# merge process parameters and features
merged = pd.merge(left=df, left_index=True,
                  right=features, right_index=True,
                  how='inner')
merged

In [ ]:
#create seperate data samples considering to determine effect of dataset size 
df.sort_values(by=['DateTime'], inplace=True)
#Case 1: All data including process parameters and features of AE signals
#df = merged.copy()

#case2: scraps only
#df_copy = df.loc[df['Durchmesser Min. Fübo'] == 1.0]

#case3: all parts without features of AE signals
#df_copy = df.copy()

#case4: Get all the first parts after dressing
#df_copy =  df.loc[df['Anzahl Teile seit letztem Abrichten Sp. 51'] == 1]
#del df_copy["Anzahl Teile seit letztem Abrichten Sp. 51"] #because this column will have only 1 value

#case5: Get all the last parts before dressing
#df_copy =  df.loc[df['Anzahl Teile seit letztem Abrichten Sp. 51'] == 32]
#del df_copy["Anzahl Teile seit letztem Abrichten Sp. 51"] #because this column will have only 1 value

#case 6: equal number of ok and scrap parts
scrap_rows = df.loc[df['Durchmesser Min. Fübo'] == 1.0]
good_parts = df.loc[df['Durchmesser Min. Fübo'] == 0.0]
good_parts = good_parts.sample(frac=0.0084, replace=True, random_state=1)
df_copy = pd.concat([scrap_rows, good_parts])

#delete these rows as they are unimportant for RCA as they don't have variance
del df_copy["DateTime"]
del df_copy["CreatedAt"]
del df_copy["Drehzahl Abrichten Sp. 51"]
del df_copy["Drehzahl Schleifen Sp. 51"]
del df_copy["Anz. Abrichthübe bis HK IO Sp. 51"]
del df_copy['Temp. Bett (B02-BT9)_mean']
del df_copy['Temp. Innenraum (B02-BT14)_mean']
del df_copy['Durchmesser Min. Fübo 1']
#del df_copy['is_scrap']

#if sample size needs to be modified
#df_copy = df_copy.sample(frac=0.1, replace=True, random_state=1)

df_copy

## Data Preprocessing

In [ ]:
#min-max normalisation
normalized_df=(df_copy-df_copy.min())/(df_copy.max()-df_copy.min())
normalized_df

In [ ]:
#discretization -- binning done (for bnlearn library) not needed for Pyagrum, as it has its own interface
df_binned = pd.DataFrame()
for col in normalized_df.columns:
    #special case where 25, 50, 75 percentile are all same
    if col == 'Anz. Abrichthübe Sp. 51':
        df_binned[col] = normalized_df[col]
        continue 
    col_desc = normalized_df.describe()[col]
    bins = [col_desc['min'], col_desc['25%'], col_desc['50%'], col_desc['75%'], col_desc['max']]
    labels = [25, 50, 75, 100]
    if np.unique(bins).size != len(bins):
        labels = labels[0:(np.unique(bins).size -1)]
    
    df_binned[col] = pd.cut(normalized_df[col], bins=bins, duplicates='drop') #, labels = labels
    
df_binned

## using Pyagrum Library

In [ ]:
#Discretization done by PyAgrum library (offers 3 hyper-parameters)
discretizer = skbn.BNDiscretizer(defaultDiscretizationMethod='uniform',
                                 defaultNumberOfBins=5,
                                 discretizationThreshold=25)

template = gum.BayesNet()
for name in normalized_df.columns:
    template.add(discretizer.createVariable(name, normalized_df[name]))

In [ ]:
#normalized and discretized data is stored in a template, this needs to stored in a csv file for using pyagrum 
normalized_df.to_csv('./Data/equal_parts_normalized_df.csv', index=False)
file_name = './Data/equal_parts_normalized_df.csv'

In [ ]:
#ground truth data created by manufacturing expert
scrap_analysis_edges = [ ('Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo', 'Unwucht Sp. 51 Leerlauf'), 
                         ('Unwucht Sp. 51 Leerlauf', 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf'), 
                         ('Unwucht Sp. 51 Leerlauf', 'Anz. Abrichthübe Sp. 51'), 
                         ('Korr. Fübo 1 Sp. 51', 'Durchmesser Min. Fübo'),
                         ('Leistung Sp. 51', 'Durchmesser Min. Fübo'), 
                         ('Taktzeit gesamt', 'Durchmesser Min. Fübo'), 
                         ('Taktzeit Abrichten OP1 Sp. 51', 'Taktzeit gesamt'), 
                         ('Anz. Abrichthübe Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'),
                         ('Korr. Fübo 1 Sp. 51', 'Ext. Durchmesser Min. Fübo 51'), 
                         ('Taktzeit Schleifen OP1 Sp. 51', 'Taktzeit gesamt'),
                         ('Be- / Entladezeit Seite A', 'Taktzeit gesamt'), ('PSH Pos. B5', 'Korr. PSH Sp. 51'), 
                         ('Ext. Korr. Fübo 1 Sp. 51', 'Durchmesser Min. Fübo'), 
                         ('Anz. Abrichthübe Sp. 51', 'Taktzeit gesamt'),
                         ('Anzahl Teile seit letztem Scheibenwechsel Sp. 51', 'Durchmesser Min. Fübo'), 
                         ('Stückzahl bis Scheibenwechsel Sp. 51', 'Durchmesser Min. Fübo'), 
                         ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Durchmesser Min. Fübo'),
                         ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Ext. Durchmesser Min. Fübo 51'),
                         ('Anzahl Teile seit letztem Scheibenwechsel Sp. 51', 'Stückzahl bis Scheibenwechsel Sp. 51'), 
                         ('Korr. Fübo 1 Sp. 51', 'Aktueller Durchmesser Sp. 51'),
                         ('Korr. Fübo 1 Sp. 51', 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf'),#check with Andre
                         ('Korr. Fübo 1 Sp. 51', 'Anzahl Teile seit letztem Abrichten Sp. 51'),
                         ('Korr. Fübo 1 Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'),
                         ('Korr. Fübo 1 Sp. 51', 'Korr. PSH Sp. 51'), ('Korr. Fübo 1 Sp. 51', 'PSH Pos. B5'),
                         ('Leistung Sp. 51', 'Peak-Beschleunigung (Zeitbereich) Sp. 51 Leerlauf'),
                         ('Anzahl Teile seit letztem Scheibenwechsel Sp. 51', 'Anz. Abrichthübe Sp. 51'),
                         ('Anzahl Teile seit letztem Scheibenwechsel Sp. 51', 'Taktzeit Abrichten OP1 Sp. 51'),
                         ('Aktueller Durchmesser Sp. 51', 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo'),
                         ('Anzahl Teile seit letztem Abrichten Sp. 51', 'Peak-Beschleunigung (Zeitbereich) WSS A Schleifen Fübo'),
                         ('Ext. Durchmesser Min. Fübo 51', 'Ext. Korr. Fübo 1 Sp. 51'),
                         ('Taktzeit Abrichten OP1 Sp. 51', 'Durchmesser Min. Fübo')
                       ]

In [ ]:
#create BN structure from ground truth data
bn=gum.BayesNet('ScrapAnalysis')
#counter = 1
nodes = []
for cols in normalized_df.columns:
    node = bn.add(gum.LabelizedVariable(str(cols), str(cols)))
    nodes.append(node)
    
import pyAgrum.lib.notebook as gnb
for link in scrap_analysis_edges:
    bn.addArc(*link)

gnb.show(bn,size="10!")

In [ ]:
#save the plot
dot_data = bng.BN2dot(bn,size="15!")
dot_data.set_size('"15,15!"')
dot_data.write_png('ground-truth.png')

## RCA model from 3 algorithms (GHC, TS, K2)

In [ ]:
#use MLFlow to save the results
with open("mlflow_logs.txt", 'w') as f:
    mlflow.start_run()
    learner = gum.BNLearner(file_name, template)

    #case 1: GreedyHillClimbing
    learner.useGreedyHillClimbing()
    bn2 = learner.learnBN()
    diff1 = compare.GraphicalBNComparator(bn,bn2).scores()
    print('Done with GHC')
    diff1['Algorithm'] = 'Greedy Hill Climbing'
    dot_data = bng.BN2dot(bn2,size="20!")
    dot_data.set_size('"20,20!"')
    dot_data.write_png('gh-bn.png')
    mlflow.log_artifact("gh-bn.png")
    
    #case 2: TabuList
    learner.useLocalSearchWithTabuList()
    bn3 = learner.learnBN()
    diff2 = compare.GraphicalBNComparator(bn,bn3).scores()
    print('Done with TS')
    diff2['Algorithm'] = 'Tabu Search'
    dot_data = bng.BN2dot(bn3,size="20!")
    dot_data.set_size('"20,20!"')
    dot_data.write_png('ts-bn.png')
    mlflow.log_artifact("ts-bn.png")
    
    #case 2: K2
    learner.useK2([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])
    bn4 = learner.learnBN()
    diff3 = compare.GraphicalBNComparator(bn,bn4).scores()
    print('Done with Algo 3')
    diff3['Algorithm'] = 'K2'
    dot_data = bng.BN2dot(bn4,size="20!")
    dot_data.set_size('"20,20!"')
    dot_data.write_png('k2-bn.png')
    mlflow.log_artifact("k2-bn.png")
    
    #concatenate all dictionaries
    dall = {}
    dall.update(diff1)
    dall.update(diff2)
    dall.update(diff3)
    mlflow.log_dict(dall, "results.txt")
    
    mlflow.end_run()

In [ ]:
#Compare ground truth and the learned BN structure from RCA models
diff1 = compare.GraphicalBNComparator(bn,bn2).dotDiff()
diff1.write_png("gt-vs-gh.png")

diff2 = compare.GraphicalBNComparator(bn,bn3).dotDiff()
diff2.write_png("gt-vs-ts.png")

diff3 = compare.GraphicalBNComparator(bn,bn4).dotDiff()
diff3.write_png("gt-vs-k2.png")


## plot confusion matrix

In [ ]:
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)
    plt.savefig('heatmap-tcdf-vs-gt.png')

In [ ]:
for diff in [diff1,diff2,diff3]:
    #get the tp, tn, fp, fn values for calculating the confusion matrix
    json_str = json.dumps(diff)

    #load the json to a string
    resp = json.loads(json_str)

    tp = resp['count']['tp']
    tn = resp['count']['tn']
    fp = resp['count']['fp']
    fn = resp['count']['fn']
    y_true = [tp, fp]
    y_pred = [fn, tn]
    labels = ["True Pos","False Neg","False Pos","True Neg"]
    categories = ["True", "False"]
    cf_matrix = np.column_stack((y_true, y_pred))

    make_confusion_matrix(cf_matrix, 
                          group_names=labels,
                          categories=categories)

## Causal inference

In [ ]:
import pyAgrum.lib.explain as explain
bn2=learner.learnBN()
print("Learned in {0}ms".format(1000*learner.currentTime()))
gnb.sideBySide(bn3,explain.getInformation(bn2))

In [ ]:
ie=gum.LazyPropagation(bn3)
ie.makeInference()
print(ie.posterior(bn3.idFromName("Durchmesser Min. Fübo")))

In [ ]:
gnb.showInference(bn3,evs={},targets={"Durchmesser Min. Fübo","Anz. Abrichthübe Sp. 51"},size="11")

In [ ]:
import pyAgrum.lib.explain as explain
explain.showInformation(bn3,{},size="14")

In [ ]:
gnb.showInference(bn2,evs={},size="15")

In [ ]:
gnb.sideBySide(bn2, gum.MarkovBlanket(bn2, 'Durchmesser Min. Fübo'), captions=["Learned Bayesian network", "Markov blanket of 'Durchmesser Min. Fübo'"])

In [ ]:
gum.MarkovBlanket(bn4, 'Durchmesser Min. Fübo')

In [ ]:
bn2.generateCPTs()

In [ ]:
learner.use3off2()
learner.useNMLCorrection()
print(learner)
ge3off2=learner.learnMixedStructure()
ge3off2

In [ ]:
learner.useMIIC()
learner.useNMLCorrection()
print(learner)
gemiic=learner.learnMixedStructure()
gemiic

In [ ]:
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb

target="Durchmesser Min. Fübo"
evs="Anz. Abrichthübe Sp. 51"

model=csl.CausalModel(bn2)
cslnb.showCausalImpact(model,target, {evs})

In [ ]:
ie=gum.LazyPropagation(bn)
ie2=gum.LazyPropagation(bn3)
p1=ie.evidenceImpact(target,[evs])
p2=gum.Potential(p1).fillWith(ie2.evidenceImpact(target,[evs]),[target,evs])
errs=((p1-p2)/p1).scale(100)
quaderr=(errs*errs).sum()
gnb.sideBySide(p1,p2,errs,quaderr,
              captions=['in original model','in learned model','relative errors','quadratic error'])